# DirectView as multiplexer

In [ ]:
import os,sys,time
import numpy as np

from IPython.core.display import display
import ipyparallel as parallel
rc = parallel.Client()

The DirectView can be readily understood as an Engine Multiplexer -
it does the same thing on all of its engines.

The only difference between running code on a single remote engine
and running code in parallel is how many engines the DirectView is
instructed to use.

You can create DirectViews by index-access to the Client.  This creates
a DirectView using the engines after passing the same index (or slice)
to the `ids` list.

In [ ]:
e0 = rc[0]
engines = rc[:]
even = rc[::2]
odd = rc[1::2]

# this is the one we are going to use:
dview = engines
dview.block = True

Now, the only difference from single-engine remote execution is that the code we run happens on all of the engines of a given view:

In [ ]:
for view in (e0, engines, even, odd):
    print(view, view.apply_sync(os.getpid))

The results of multiplexed execution is always a list of the length of the number of engines.

In [ ]:
engines['a'] = 5
engines['a']

# Scatter and Gather

Lots of parallel computations involve partitioning data onto processes.  
DirectViews have `scatter()` and `gather()` methods, to help with this.
Pass any container or numpy array, and IPython will partition the object onto the engines wih `scatter`,
or reconstruct the full object in the Client with `gather()`.

In [ ]:
dview.scatter('a', list(range(16)))
dview['a']

In [ ]:
dview.gather('a')

In [ ]:
dview.execute("asum = sum(a)")
dview.gather('asum')

We can pass a 'flatten' keyword,
to instruct engines that will only get one item of the list to
get the actual item, rather than a one-element sublist:

In [ ]:
dview.scatter('id', rc.ids)
dview['id']

In [ ]:
dview.scatter('id', rc.ids, flatten=True)
dview['id']

Scatter and gather also work with numpy arrays

In [ ]:
A = np.random.randint(1,10,(16,4))
B = np.random.randint(1,10,(4,16))
display(A)
display(B)

In [ ]:
dview.scatter('A', A)
dview.scatter('B', B)
display(e0['A'])
display(e0['B'])

## Excercise: Parallel Matrix Multiply

Can you compute the Matrix product `C=A.dot(B)` in parallel? (not looking for brilliant, just correct).


In [ ]:
%run ../hints
mmhint()

In [ ]:
%load ../soln/matmul.py

Let's run this, and validate the result against a local computation.

In [ ]:
C_ref = A.dot(B)
C1 = pdot(dview, A, B)
# validation:
(C1==C_ref).all()

# Map

DirectViews have a map method, which behaves just like the builtin map,
but computed in parallel.

In [ ]:
dview.block = True

serial_result   =  list(map(lambda x:x**10, range(32)))
parallel_result = dview.map(lambda x:x**10, range(32))

serial_result == parallel_result

`DirectView.map` partitions the sequences onto each engine,
and then calls `map` remotely.  The result is always a single
IPython task per engine.

In [ ]:
amr = dview.map_async(lambda x:x**10, range(32))
amr.msg_ids

In [ ]:
amr = dview.map_async(lambda x:x**10, range(3200))
amr.msg_ids

### The motivating example

In [ ]:
from IPython.display import display, Image
%run ../images_common

In [ ]:
pictures = get_pictures(os.path.join('..', 'images', 'castle'))

In [ ]:
%px cd {os.getcwd()}

In [ ]:
%%px
%matplotlib inline
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage import measure

In [ ]:
engines.push(dict(
    plot_corners=plot_corners,
    find_corners=find_corners,
))

In [ ]:
ar = e0.apply_async(get_corners_image, pictures[0])
ar.wait_interactive()
Image(data=ar.get())

In [ ]:
amr = engines.map_async(get_corners_image, pictures[:len(engines)])
amr.wait_interactive()
for pngdata in amr:
    display(Image(data=pngdata))

## Exercises and Examples

- [Remote Iteration](../exercises/Remote%20Iteration.ipynb)
- [Monte Carlo π](../exercises/Monte%20Carlo%20π.ipynb)

# Moving on

IPython.parallel can also be used for [load-balanced execution](Load-Balancing.ipynb),
when you just want code to run, but don't care where.